In [3]:
from sly import Lexer, Parser
import sys

# Implementacion descendente:

In [ ]:
#Lexer:
class CalcLexer(Lexer):
    tokens={NUM}
    literals={';','+','-','*','/','(',')'}
    NUM=r'[0-9]+'
    ignore=r'[ \t]'
    
    @_(r'[0-9]+')
    def NUM(sef, f):
        f.value=int(f.value)
        return f
    
    def error(self, f):
        print('Bad character %r' % f.value[0])
        sys.exit(0)

In [15]:
#Parser:
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
        
    @_('entrada expr ";"')
    def entrada(self, f):
        print('El valor de la expresion es: %d' %f.expr)
        
    @_(' ')
    def entrada(self, f):
        pass
    
    @_('expr "+" suma')
    def expr(self, f):
        return f.expr+f.suma
    
    @_('expr "-" suma')
    def expr(self, f):
        return f.expr-f.suma
    
    @_('suma')
    def expr(self, f):
        return f.suma
        
    @_('suma "*" factor')
    def suma(self, f):
        return f.suma*f.factor
        
    @_('suma "/" factor')
    def suma(self, f):
        return f.suma/f.factor
    
    @_('factor')
    def suma(self, f):
        return f.factor
    
    @_('"-" factor')
    def factor(self, f):
        return -f.factor
    
    @_("NUM")
    def factor(self, f):
        return f.NUM
    
    @_('"(" expr ")"')
    def factor(self, f):
        return f.expr

In [ ]:
#Main:
if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()
    while True:
        try:
            text = input('one expression plix > ')
        except EOFError:
            break
        if text:
            parser.parse(lexer.tokenize(text))

# With AST:

In [5]:
from sly import Lexer, Parser
import sys

In [6]:
class Nodo():
    pass
class NodoNormal(Nodo):
    def escribe(a):
        print("parriba %d"%a)
        return a
        
class NodoSuma(Nodo):
    def escribe(a,b):
        print("%d + %d "%(a,b))
        return a+b
    
class NodoResta(Nodo):
    def escribe(a,b):
        print("%d - %d "%(a,b))
        return a-b
    
class NodoMulti(Nodo):
    def escribe(a,b):
        print("%d * %d "%(a,b))
        return a*b
class NodoDiv(Nodo):
    def escribe(a,b):
        print("%d / %d"%(a,b))
        return a/b
        
class NodoUnario(Nodo):
    def escribe(a):
        print("-%d"%a)
        return -a

In [7]:
#Lexer:
class CalcLexer(Lexer):
    tokens={NUM}
    literals={';','+','-','*','/','(',')'}
    NUM=r'[0-9]+'
    ignore=r'[ \t]'
    
    @_(r'[0-9]+')
    def NUM(sef, f):
        f.value=int(f.value)
        return f
    
    def error(self, f):
        print('Bad character %r' % f.value[0])
        sys.exit(0)

In [8]:
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
        
    @_('entrada expr ";"')
    def entrada(self, f):
        print('El valor de la expresion es: %d' %f.expr)
        
    @_(' ')
    def entrada(self, f):
        pass
    
    @_('expr "+" suma')
    def expr(self, f):
        return NodoSuma.escribe(f.expr,f.suma)
    
    @_('expr "-" suma')
    def expr(self, f):
        return NodoResta.escribe(f.expr,f.suma)
    
    @_('suma')
    def expr(self, f):
        return NodoNormal.escribe(f.suma)
        
    @_('suma "*" factor')
    def suma(self, f):
        return  NodoMulti.escribe(f.suma,f.factor)
        
    @_('suma "/" factor')
    def suma(self, f):
        return NodoDiv.escribe(f.suma,f.factor)
    
    @_('factor')
    def suma(self, f):
        return NodoNormal.escribe(f.factor)
    
    @_('"-" factor')
    def factor(self, f):
        return NodoUnario.escribe(f.factor)
    
    @_("NUM")
    def factor(self, f):
        return NodoNormal.escribe(f.NUM)
    
    @_('"(" expr ")"')
    def factor(self, f):
        return NodoNormal.escribe(f.expr)

In [ ]:
#Main:
if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()
    while True:
        try:
            text = input('one expression plix > ')
        except EOFError:
            break
        if text:
            parser.parse(lexer.tokenize(text))

# Implementacion Ascendente:

In [ ]:
from sly import Lexer, Parser
import sys

In [9]:
tabla = {}

class Nodo():
    def escribir(self):
        pass

class NodoEntero(Nodo):
    def escribir(self):
        return 'Entero'

class NodoFlotante(Nodo):
    def escribir(self):
        return 'Flotante'

class NodoCaracter(Nodo):
    def escribir(self):
        return 'Caracter'

class NodoPuntero(Nodo):
    def __init__(self,n):
        self.n = n
    def escribir(self):
        return "Puntero a " + self.n.escribir()

In [10]:
class CalcLexer(Lexer):
    tokens={ID, CHAR, INTEGER, FLOAT}
    ignore=' \t'
    literals={'*',',',';'}
    #Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    ID['char']=CHAR
    ID['integer']=INTEGER
    ID['float']=FLOAT

    @_(r'\n+')
    def newline(self, t):
        self.lineno+=t.value.count('\n')

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index+=1

In [11]:
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
    
    @_('defi entrada')
    def entrada(self, p):
        pass
    
    @_(' ')
    def entrada(self, p):
        pass

    @_('tipo lista ";"')
    def defi(self, p):
        return p.tipo
    
    @_('INTEGER')
    def tipo(self, p):
        return NodoEntero()

    @_('FLOAT')
    def tipo(self, p):
        return NodoFlotante()

    @_('CHAR')
    def tipo(self, p):
        return NodoCaracter()
    
    @_(' empty5 elm empty4 resto')
    def lista(self, p):
        return p[-2]

    @_('')
    def empty4(self,p):
        return p[-3]

    @_('')
    def empty5(self,p):
        return p[-1]
    
    @_('"," empty3 elm empty2 resto')
    def resto(self, p):
        return p[-4]

    @_('')
    def empty2(self,p):
        return p[-4]

    @_('')
    def empty3(self,p):
        return p[-2]

    @_('')
    def resto(self, p):
        return p[-1]

    @_('"*" empty1 elm')
    def elm(self, p):
        return p.empty1

    @_('')
    def empty1(self,p):
        return NodoPuntero(p[-2])

    @_('ID')
    def elm(self, p):
        tabla[p.ID] = p[-2].escribir()    

In [ ]:
if __name__ == '__main__':
    lexer=CalcLexer()
    parser=CalcParser()
    while True:
        try:
            text = input('data type list > ')
        except EOFError:
            break
        if text:
            parser.parse(lexer.tokenize(text))
        print(tabla)